In [22]:
import qiskit
import covalent as ct

In [23]:
IBMQ.save_account('YOUR_API_KEY')

configrc.store_credentials:WARNING:2023-01-29 00:18:47,662: Credentials already present. Set overwrite=True to overwrite.


In [24]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute
import numpy as np

@ct.electron
def construct_circuit():
    q_register = QuantumRegister(7)
    c_register = ClassicalRegister(7)
    
    circuit = QuantumCircuit(7)
    
    unitary_operators = [circuit.X, circuit.Z]
    for operator in unitary_operators:
        circuit.unitary(operator)
    
    circuit.measure(q_register, c_register)
    
    return circuit

@ct.electron
def simulate(circuit):
    from qiskit.providers.ibmq import IBMQ
 
    provider = IBMQ.get_provider(hub='ibm-q-community', group='mit-hackathon', project='main')
    provider.backends()

    backend = provider.get_backend('ibm_oslo')
    
    from qiskit.providers.aer import AerSimulator

    # Generate a simulator the mimics the real quantum system with the latest calibration
    backend_sim = AerSimulator.from_backend(backend)
    
@ct.lattice
def test():
    circuit = construct_circuit()
    return simulate(circuit=circuit)